In [1]:
%cd ../..

C:\Users\Wight\PycharmProjects\ThesisMain


In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from collections import OrderedDict


from PIL import Image
import os
from models.CSRNet.CSRNet import CSRNet
from models.CSRNet.CSRNet_functional import CSRNet_functional

import importlib
import time

# Settings and Parameters
Here we define the DeiT model that we wish to evaluate and the corresponding parameters to evaluate it.

In [3]:
# trained_model_path = 'notebooks\\TL\\save_state_ep_560_new_best_MAE_7.454.pth'  # The path to trained model file (something like XYZ.pth)
trained_model_path = 'D:\\OneDrive\\OneDrive - UvA\\ThesisData\\trained_models\\CSRNet ML WE Oneshot\\save_state_ep_2725_new_best_MAE_4.841.pth'  # The path to trained model file (something like XYZ.pth)

label_factor = 100  # The label factor used to train this specific model.
dataset = 'WE_CSRNet_Meta'  # Must be the exact name of the dataset
save_results = True  # When true, save the images, GTs and predictions. A folder for this is created automatically.
set_to_eval = 'test'  # val', 'test'. Which split to test the model on. 'train' does not work!

# all_adapt_imgs = [
#     [
#         ['104207_1-04-S20100821071000000E20100821120000000_007550.jpg'], 
#         ['200608_C08-02-S20100626083000000E20100626233000000_clip1_021050.jpg'], 
#         ['200702_C09-01-S20100717083000000E20100717233000000_007550.jpg'],
#         ['202201_1-01-S20100922060000000E20100922235959000_clip1_004550.jpg'],
#         ['500717_D11-03-S20100717083000000E20100717233000000_016550.jpg']
#     ],
#     [
#         ['104207_1-04-S20100821071000000E20100821120000000_172550.jpg'],
#         ['200608_C08-02-S20100626083000000E20100626233000000_clip1_061550.jpg'],
#         ['200702_C09-01-S20100717083000000E20100717233000000_141050.jpg'],
#         ['202201_1-01-S20100922060000000E20100922235959000_clip1_023450.jpg'],
#         ['500717_D11-03-S20100717083000000E20100717233000000_142550.jpg']
#     ],
#     [
#         ['104207_1-04-S20100821071000000E20100821120000000_078050.jpg'],
#         ['200608_C08-02-S20100626083000000E20100626233000000_clip1_159050.jpg'],
#         ['200702_C09-01-S20100717083000000E20100717233000000_091550.jpg'],
#         ['202201_1-01-S20100922060000000E20100922235959000_clip1_064850.jpg'],
#         ['500717_D11-03-S20100717083000000E20100717233000000_144050.jpg']
#     ]
# ]

adapt_imgs = [
    [
        '104207_1-04-S20100821071000000E20100821120000000_007550.jpg',
        '104207_1-04-S20100821071000000E20100821120000000_090050.jpg',
        '104207_1-04-S20100821071000000E20100821120000000_172550.jpg',
        '104207_1-04-S20100821071000000E20100821120000000_069050.jpg',
        '104207_1-04-S20100821071000000E20100821120000000_078050.jpg'
        
    ],
    [
        '200608_C08-02-S20100626083000000E20100626233000000_clip1_021050.jpg',
        '200608_C08-02-S20100626083000000E20100626233000000_clip1_169550.jpg',
        '200608_C08-02-S20100626083000000E20100626233000000_clip1_061550.jpg',
        '200608_C08-02-S20100626083000000E20100626233000000_clip1_091550.jpg',
        '200608_C08-02-S20100626083000000E20100626233000000_clip1_159050.jpg'
        
    ],
    [
        '200702_C09-01-S20100717083000000E20100717233000000_007550.jpg',
        '200702_C09-01-S20100717083000000E20100717233000000_141050.jpg',
        '200702_C09-01-S20100717083000000E20100717233000000_003050.jpg',
        '200702_C09-01-S20100717083000000E20100717233000000_076550.jpg',
        '200702_C09-01-S20100717083000000E20100717233000000_091550.jpg'
        
    ],
    [
        '202201_1-01-S20100922060000000E20100922235959000_clip1_004550.jpg',
        '202201_1-01-S20100922060000000E20100922235959000_clip1_041450.jpg',
        '202201_1-01-S20100922060000000E20100922235959000_clip1_023450.jpg',
        '202201_1-01-S20100922060000000E20100922235959000_clip1_011750.jpg',
        '202201_1-01-S20100922060000000E20100922235959000_clip1_064850.jpg'
    ],
    [
        '500717_D11-03-S20100717083000000E20100717233000000_016550.jpg',
        '500717_D11-03-S20100717083000000E20100717233000000_057050.jpg',
        '500717_D11-03-S20100717083000000E20100717233000000_142550.jpg',
        '500717_D11-03-S20100717083000000E20100717233000000_102050.jpg',
        '500717_D11-03-S20100717083000000E20100717233000000_144050.jpg'
    ]
]

# Prepare for evaluation
Use the settings to load the DeiT model and dataloader for the test set. Also loads the transform with which we can restore the original images. Cuda is required!
If save_results is True, also create the directory in which the predictions are saved.

In [4]:
loss_fn = torch.nn.MSELoss()

In [5]:
dataloader = importlib.import_module(f'datasets.meta.{dataset}.loading_data').loading_data
cfg_data = importlib.import_module(f'datasets.meta.{dataset}.settings').cfg_data

train_loaders, val_loaders, test_loaders, restore_transform = dataloader(adapt_imgs)
if set_to_eval == 'val':
    my_dataloaders = val_loaders
elif set_to_eval == 'test':
    my_dataloaders = test_loaders
else:
    print(f'Error: invalid set --> {set_to_eval}')

83 train scenes found.
114 test images found.
115 test images found.
115 test images found.
115 test images found.
115 test images found.


In [6]:
def load_model_and_optim():
    model = CSRNet()
#     model.make_alpha(42)  # Whatever, will be overwritten anyway
    model.cuda()
    state_dict = torch.load(trained_model_path)
    model.make_alpha(42)  # Whatever, will be overwritten anyway
    net = state_dict['net']
    model.load_state_dict(net)

#     model.make_alpha(42)  # Whatever, will be overwritten anyway
    model.cuda()
    
    model_functional = CSRNet_functional()
    model.eval()
        
    return model, model_functional

In [7]:
save_dir = None
if save_results:
    save_folder = 'CSRNet_meta' + '_' + dataset + '_' + set_to_eval + '_' + time.strftime("%m-%d_%H-%M", time.localtime())
    save_dir = os.path.join('notebooks', save_folder)  # Manually change here is you want to save somewhere else
    os.mkdir(save_dir)

# Evaluation loop and save funtion

In [8]:
def eval_on_scene(model_funct, model_weights, scene_dataloader):
    model.eval()
    with torch.no_grad():
        preds = []
        gts = []
        AEs = []  # Absolute Errors
        SEs = []  # Squared Errors

        for idx, (img, gt) in enumerate(scene_dataloader):
            img = img.cuda()
           
            den = model_functional.forward(img, model_weights, training=False)  # Precicted density crops
            den = den.cpu()

            gt = gt.squeeze()  # Remove channel dim
            den = den.squeeze()  # Remove channel dim
            
#             img = restore_transform(img.squeeze())  # Original image
            pred_cnt = den.sum() / cfg_data.LABEL_FACTOR
            gt_cnt = gt.sum() / cfg_data.LABEL_FACTOR
            
            preds.append(pred_cnt.item())
            gts.append(gt_cnt.item())
            AEs.append(torch.abs(pred_cnt - gt_cnt).item())
            SEs.append(torch.square(pred_cnt - gt_cnt).item())
            relative_error = AEs[-1] / gt_cnt * 100
#             print(f'IMG {idx:<3} '
#                   f'Prediction: {pred_cnt:<9.3f} '
#                   f'GT: {gt_cnt:<9.3f} '
#                   f'Absolute Error: {AEs[-1]:<9.3f} '
#                   f'Relative Error: {relative_error:.1f}%')
            
#             if save_path:
#                 plot_and_save_results(save_path, img, idx, gt, den, pred_cnt, gt_cnt)
            
        MAE = np.mean(AEs)
        MSE = np.sqrt(np.mean(SEs))

    return preds, gts, MAE, MSE

In [9]:
def adapt_to_scene(model, scene_dataloader):
    model.train()
    
    theta = OrderedDict((name, param) for name, param in model.named_parameters())
    theta_values = list(theta[k] for k in theta if not k.startswith('alpha.'))
    theta_names = list(k for k in theta if not k.startswith('alpha.'))
    alpha_values = list(theta[k] for k in theta if k.startswith('alpha.'))


    adapt_img, adapt_gt = scene_dataloader.dataset.get_adapt_batch()
    adapt_img = adapt_img.cuda()
    adapt_gt = adapt_gt.squeeze().cuda()

    pred = model_functional.forward(adapt_img, theta, training=True)
    pred = pred.squeeze()

    loss = loss_fn(pred, adapt_gt)
    grads = torch.autograd.grad(loss, theta_values)

    theta_prime = OrderedDict(
        (n, w - a * g) for n, w, a, g in zip(theta_names, theta_values, alpha_values, grads)
    )
    
    return theta_prime

In [10]:
def save_scene_graph(preds_before, preds_after, gts, save_name):
    MAE_before = np.mean(np.abs(np.array(preds_before) - np.array(gts)))
    MAE_after = np.mean(np.abs(np.array(preds_after) - np.array(gts)))
    save_path = os.path.join(save_dir, save_name)
    xs = np.arange(len(gts))
    plt.figure(figsize=(20,10))
    plt.title(f'MAE before: {MAE_before:.3f}, MAE after: {MAE_after:.3f}, MAE improvement: {MAE_before - MAE_after:.3f}')
    plt.plot(xs, gts, color='green', label='GT')
    plt.plot(xs, preds_before, color='blue', label='Before')
    plt.plot(xs, preds_after, '--', color='red', label='After')
    plt.legend()
#     plt.savefig(save_path)
    plt.show()

In [11]:
# model, model_functional = load_model_and_optim()  # Learning rate is not used when not adapting
# theta = OrderedDict((name, param) for name, param in model.named_parameters())

# scene_dataloaders, restore_transform, cfg_data = get_dataloaders(None)
# for idx, scene_dataloader in enumerate(scene_dataloaders):
#     print(f'scene {idx + 1}')
#     preds_before, gts, MAE_before, MSE_before = eval_on_scene(model_functional, theta, scene_dataloader)
#     print(f'  No adapt MAE: {MAE_before:.3f}, MSE: {MSE_before:.3f}')
  

In [12]:
for scene_idx in range(5):
    print(f'Scene {scene_idx + 1}')

    scene_dataloader = my_dataloaders[scene_idx]

    model, model_functional = load_model_and_optim()
    theta = OrderedDict((name, param) for name, param in model.named_parameters())

    preds_before, gts, MAE_before, MSE_before = eval_on_scene(model_functional, theta, scene_dataloader)
    print(f'    Before adapt MAE/MSE: {MAE_before:.3f}/{MSE_before:.3f}')
#         continue
    adapted_weights = adapt_to_scene(model, scene_dataloader)

    preds_after, gts, MAE_after, MSE_after = eval_on_scene(model_functional, adapted_weights, scene_dataloader)
    print(f'    After adapt MAE/MSE: {MAE_after:.3f}/{MSE_after:.3f}')

    save_scene_graph(preds_before, preds_after, gts, f'CSRNet_WE_ML_FS_scene_{scene_idx + 1}.jpg')

Scene 1


C:\Users\Wight\Anaconda3\envs\ThesisMain\lib\site-packages\torch\nn\modules\container.py:552: UserWarning: Setting attributes on ParameterDict is not supported.
  warnings.warn("Setting attributes on ParameterDict is not supported.")


    Before adapt MAE/MSE: 1.775/2.293


RuntimeError: CUDA out of memory. Tried to allocate 64.00 MiB (GPU 0; 4.00 GiB total capacity; 2.66 GiB already allocated; 58.10 MiB free; 2.76 GiB reserved in total by PyTorch)

In [13]:
# a = [1, 2, 4]
# b = [2, 1, 0]
# np.mean(np.abs(np.array(a) - np.array(b)))

In [14]:
# save_scene_graph(preds_before, preds_after, gts, f'scene_{scene_idx + 1}_img_crowdedness_{crowdedness_names[idx]}.jpg')

In [15]:
a = 12.3
print(f'asdasd: {a:7.3f}')

asdasd:  12.300
